<a href="https://colab.research.google.com/github/carlos-alves-one/-Amazon-Review-NLP/blob/main/Sentiment_Analysis_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Goldsmiths University of London
### MSc. Data Science and Artificial Intelligence
### Module: Natural Language Processing
### Author: Carlos Manuel De Oliveira Alves
### Student: cdeol003
### Coursework Project

# Data Collection

### Load the data

In [1]:
# Imports the 'drive' module from 'google.colab' and mounts the Google Drive to
# the '/content/drive' directory in the Colab environment.
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


Dataset source: https://www.kaggle.com/datasets/akudnaver/amazon-reviews-dataset

License: Unknown

In [2]:
# Import the pandas library and give it the alias 'pd' for data manipulation and analysis
import pandas as pd

# Load the dataset Amazon Review Details from Google Drive
data_path = '/content/drive/MyDrive/amazon_project/amazon-review-details.csv'
amazon_data = pd.read_csv(data_path)

# Display the first few rows of the dataframe
amazon_data.head(3).T


,0,1,2
report_date,2019-01-02,2019-01-03,2019-01-03
online_store,FRESHAMAZON,FRESHAMAZON,FRESHAMAZON
upc,8718114216478,5000184201199,5000184201199
retailer_product_code,B0142CI6FC,B014DFNNRY,B014DFNNRY
brand,Dove Men+Care,Marmite,Marmite
category,Personal Care,Foods,Foods
sub_category,Deos,Savoury,Savoury
product_description,Dove Men+Care Extra Fresh Anti-perspirant Deod...,Marmite Spread Yeast Extract 500g,Marmite Spread Yeast Extract 500g
review_date,2019-01-01,2019-01-02,2019-01-02
review_rating,5,5,4


In [3]:
# Extracting the first review text from the 'review_text' column of the amazon_data dataframe
data = amazon_data['review_text'].values[0]
data


'As you get older, you know what you like and what is suitable for your body. I like all Dove products. Gives you that fresh all over, wide awake feeling and no dandruff or flakey skin. No smelly a/pits!'

# Data Selection

In [19]:
# Selecting only the 'review_text' and 'review_rating' columns
new_dataset = amazon_data[['review_text', 'review_rating']]

# Define the path for the new CSV file
output_file_path = '/content/drive/MyDrive/amazon_project/filtered_reviews.csv'

# Save the new dataset to a CSV file
new_dataset.to_csv(output_file_path, index=False)


In [21]:
# Load the dataset Amazon Review with the columns relevant from Google Drive
data_path = '/content/drive/MyDrive/amazon_project/filtered_reviews.csv'
amazon_data = pd.read_csv(data_path)

# Display the first few rows of the dataframe
amazon_data.head(3)


,review_text,review_rating
0,"As you get older, you know what you like and w...",5
1,Three gigantic marmite jars that will last pro...,5
2,Excellent,4
